# Python爬虫

## 基本流程
* **准备工作**
  
  通过浏览器查看要分析的目标网页
* **获取数据**
  
  通过http库向目标站点发起请求，请求可以包含额外的header等信息。如果服务器能正常响应，会得到一个response，即为所要获取的页面内容
* **解析内容**
  
  得到的内容可能是HTML、json等格式，可以用页面解析库、正则表达式等进行解析。
* **保存数据**
  
  保存形式多样，可以存为文本，也可以保存到数据库，或者保存特定格式的文件。

## 一、准备工作
首先安装lxml、beautifulsoup包
win+r打开cmd窗口
输入
```
pip install beautifulsoup4
pip install lxml 
```
打开[豆瓣阅读网站](https://read.douban.com/provider/all)，用F12查看网站源码。

## 二、获取数据

* **request.get()** 发送http get请求，返回响应操作
  * 需要 `import requests`库
  * response对象常用方法：
    ```
    r.headers
    #http响应内容的头部内容，来返回get请求获得网页的头部信息。
    r.status_code
    #http请求的返回状态，200表示连接成功，404表示连接失败
    r.text
    #http响应内容的字符串形式，url对应的页面内容
    r.encoding
    #从HTTP header中猜测的响应内容编码方式
    r.apparent_encoding
    #从内容分析出的响应内容的编码方式（备选编码方式）
    r.content
    #HTTP响应内容的二进制形式
    ```

* 用**BeautifulSoup**从HTML/XML中提取数据
  
  * BeautifulSoup(markup, "html.parser")或BeautifulSoup(markup, "lxml")，推荐使用lxml作为解析器,因为效率更高。
  *  find_next()：用于查找符合查询条件的第一 个标签节点。
  *  find_all():查找所有符合查询条件的标签节点，并返回一个列表。
  *  
     ```
     //将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
     sp = BeautifulSoup(response.text,'lxml') 
      ```



* **headers和cookies的关系**

  怎么说呢，网站是欢迎真人访问的，这会提高网站的流量和热度等等，但是网站不欢迎机器人，机器人做的事情对于网站
  来说是无利可图的。大家应该已经猜到了，这个机器人就是我们的爬虫，而这个借用法官的名声就是headers；后面的那些法官凭证就是网站为了安全设置的cookies。
* **如何通过浏览器获取网页的cookie**
  * 打开要爬取的网页
  * 打开开发者模式，按F12
  * 点击左侧框的all文件
  * 在上方元素/控制../网络/ 中找到网络一栏
  * 在右侧找到cookie并复制
* **find()和find_next()辨析**
  
  * find()和find_next()都只搜索传入区域内的板块
  * find()返回传入区域要搜索的第一个位置。find_next()返回传入区域的下一个位置
    eg.当传入区域li_s含多个div时，li_s.find('div')永远返回第一个。用li_s.find_next('div').find_next('div')可以找到第二个div


In [2]:
import sys
import numpy as np
import json
import re
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import base64
import os
import random
import time
#定义代理池
proxy_list = [
    '127.0.0.1:15732',
    '192.168.56.1.15732'
]
user_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
                   'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0'
                   ]
proxy = random.choice(proxy_list)
cookie={'cookie':'bid=kRRUP5Adrsc; _ga=GA1.3.1583431493.1679359048; _gid=GA1.3.240421151.1679359048; _ga=GA1.1.1583431493.1679359048; page_style="mobile"; dbcl2="215291240:+lGgZ069L0g"; _pk_ses.100001.a7dd=*; ck=AT7V; _ga_RXNMP372GL=GS1.1.1679406549.4.1.1679408190.60.0.0; _pk_id.100001.a7dd=0f38c905a23f4f70.1679359049.4.1679408190.1679402067.; _gat=1'}
proxies = {
'http':  proxy,
'https':  proxy,
}

headers = { 
        'User-Agent': random.choice(user_agents)
    }
url='https://read.douban.com/provider/all' 
res = requests.get(url,headers=headers,cookies=cookie,proxies=proxies)
sp = BeautifulSoup(res.text,'lxml') 

pb=sp.find('div',{'class':'provider-group'})
#print(pb)

pbs=sp.select('.provider-group')
pbs=sp.find_all('div',{'class':'provider-group'})
print(len(pbs))

ProxyError: HTTPSConnectionPool(host='read.douban.com', port=443): Max retries exceeded with url: /provider/all (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000019798EB5400>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')))

In [3]:
import sys
sys.path.append('/home/aistudio/external-libraries')
import json
import re
import requests
import datetime
import time
from bs4 import BeautifulSoup
import os
import requests,random
#定义代理池
proxy_list = [
    '127.0.0.1:15732',
    '192.168.56.1.15732'
]
user_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
                   'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0'
                   ]
proxy = random.choice(proxy_list)

proxies = {
'http':  proxy,
'https':  proxy,
}

headers = { 
        'User-Agent': random.choice(user_agents)
    }
url='https://read.douban.com/provider/all'                        
cookie={'cookie':'bid=kRRUP5Adrsc; _ga=GA1.3.1583431493.1679359048; _gid=GA1.3.240421151.1679359048; _ga=GA1.1.1583431493.1679359048; page_style="mobile"; dbcl2="215291240:+lGgZ069L0g"; _pk_ses.100001.a7dd=*; ck=AT7V; _ga_RXNMP372GL=GS1.1.1679406549.4.1.1679408190.60.0.0; _pk_id.100001.a7dd=0f38c905a23f4f70.1679359049.4.1679408190.1679402067.; _gat=1'}
try:
    response = requests.get(url,headers=headers,proxies=proxies)
    #将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
    soup = BeautifulSoup(response.text,'lxml')  
    
    #返回所有的<table>所有标签
    #
    publishes = soup.find_all('div',{'class':'provider-group'})
    print(len(publishes))
    pbs=[]
    for index,pb in enumerate(publishes):
        #if (index<=1):
        if True:
            pb_list={}
            pb_list['item_name']=pb.find_next('div').text
            item_list=[]
            p=pb.find_next('ul')
            li_s=p.find_all('li')
            
    #print(l[0])
            for li in li_s:
                item_li={}
                item_li['href']='https://read.douban.com'+li.find_next('a').get('href')
                #item_li['others']=li.find_next('div').find_next('div').find_next_sibling('div')
                print(li.find_next('div',{'class':'name'}))
                item_detail=li.find_next('div').find_next('div').find_next_sibling('div')
                #print(item_detail)
                pb_name=item_detail.find_next('div')
                item_li['pb_name']=pb_name.text
                #print(pb_name.text)
                pb_count_str=pb_name.find_next_sibling('div').text
                idx=pb_count_str.find(' ')
                pb_count_str=pb_count_str[0:idx]
                #print(pb_count_str)
                item_li['pb_count']= pb_count_str
                
                item_list.append([pb_count_str,pb_name.text])
                print(li.find_next('div').find_next('div').find_next_sibling('div'))  
                #print(li.find_next('div').find_next_sibling('div'))  
                time.sleep(0.2)
            pb_list['item_list']=item_list
            #print(pb_list)
            pbs.append(pb_list)
    #print(len(pbs))
    #json_data = json.loads(str(pbs)) 
    json_data = json.loads(str(pbs).replace("\'","\"").replace("\\xa0",""))  
    with open('work/' + 'pbs.json', 'w', encoding='UTF-8') as f:
        json.dump(json_data, f, ensure_ascii=False)
  
    
except Exception as e:
    print(e)

HTTPSConnectionPool(host='read.douban.com', port=443): Max retries exceeded with url: /provider/all (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000197B22DE490>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。')))


In [ ]:
with open('work/pbs.json', 'r', encoding='UTF-8') as file:
         json_array = json.loads(file.read())

In [12]:
import sys
import numpy as np
sys.path.append('/home/aistudio/external-libraries')
import json
import re
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import base64
import os
import random
import time
proxy_list = [
    '127.0.0.1:15732',
    '192.168.56.1.15732'
]
proxy = random.choice(proxy_list)

proxies = {
'http':  proxy,
'https':  proxy,
}
user_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
                   'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0'
                   ]
headers = { 
       'User-Agent': random.choice(user_agents),
}
url='https://read.douban.com/provider/all'                     
cookie={'cookie':'bid=QhkSDkC-6D4; dbcl2="269058597:eXNEBe+CNi8"; ll="108235"; __utma=30149280.1604062177.1679555773.1679555773.1679555773.1; __utmz=30149280.1679555773.1.1.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __gads=ID=8a3e8dcdd0ebd524-22aef64e8edc0077:T=1679555778:RT=1679555778:S=ALNI_MYat1Lbo2a4rKijb4czFgslEoxqvw; _ga=GA1.3.1604062177.1679555773; _gid=GA1.3.939193650.1679555779; _ga=GA1.1.1604062177.1679555773; _pk_ref.100001.a7dd=%5B%22%22%2C%22%22%2C1679633561%2C%22https%3A%2F%2Fmovie.douban.com%2F%22%5D; _pk_ses.100001.a7dd=*; ck=MFsm; __gpi=UID=00000bdebf1cdfd9:T=1679555778:RT=1679633581:S=ALNI_Mb7pC52M6-drvqg3ia2C6iRFXIesA; _gat=1; _pk_id.100001.a7dd=58088c42b0c59ce5.1679555779.2.1679633623.1679555977.; _ga_RXNMP372GL=GS1.1.1679633560.3.1.1679633623.59.0.0'}
try:
    response = requests.get(url,headers=headers,cookies=cookie,proxies=proxies)
    #将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
    soup = BeautifulSoup(response.text,'lxml')  
    
    #返回所有的<div>所有标签
    publishes = soup.find_all('div',{'class':'provider-group'})
    pbs=[]
    item_list=[]
    #print(publishes)
    #enumerate爬虫中的遍历
    #遍历所有出版社
    #pb为当前出版社
    for index,pb in enumerate(publishes):
        #if (index<=1):
        if True:
            pb_list={}
            pb_list['item_name']=pb.find_next('div').text
            p=pb.find_next('ul')
            li_s=p.find_all('li')#li_s存储了当前出版社的所有数据。
            
        #print(li_s)
        for li in li_s:
            item_li={}
            #item_li为进入当前出版社内页书单的链接
            item_li['href']='https://read.douban.com'+li.find_next('a').get('href')

            url2=item_li['href']
            response2=requests.get(url2,headers=headers,cookies=cookie,proxies=proxies)
            soup2 = BeautifulSoup(response2.text,'lxml')  
            
            #遍历当前出版社的所有书单页面
            while soup2.find('li',class_='next')!=None:
                #booklist为当前页面的所有<div class=info>的书籍数据
                booklist=soup2.find_all('div',{'class':'info'})
                #print(booklist)
                #print(publishes2)
                #遍历当前页面的所有书籍
                #book为当前的书籍数据
                for book in booklist:
                    
                    if(book.find('h4',class_='title')==None):
                        continue
                    title =book.find('h4',class_='title').text
                    item_li['name']=title
                    if(book.find('div',class_='sales-price')!=None):
                        price=book.find('div',class_='sales-price').text
                        item_li['price']=price
                    elif(book.find('span',class_='discount-price')!=None):
                        price=book.find('span',class_='discount-price').text
                        item_li['price']=price
                    elif(book.find('span',class_='price-tag')!=None):
                        price=book.find('span',class_='price-tag').text
                        item_li['price']=price
                    else:
                        continue
                    #输出查看
                    print(f"《{title}》:{price}")#1000行截断，保存成xlsx比较好
                    #item_list用来存储要求得的书名和价格的list型数据结构，一维
                    item_list.append([title,price])
                    t = random.random() #随机大于0 且小于1 之间的小数
                    time.sleep(t)
                    
                temp2=soup2.find('li',class_='next')
                #若存在后页
                if temp2.find('a')!=None:
                    #跳转到下一页
                    url3=url2+temp2.find('a').get('href')
                    response2=requests.get(url3,headers=headers,cookies=cookie,proxies=proxies)
                    soup2 = BeautifulSoup(response2.text,'lxml') 
                else:
                    break
                
    df=pd.DataFrame(item_list)
    df.columns=['书籍名称','价格']
    print(df)
    #保存到excel文件中
    df.to_excel("爬虫数据.xlsx")
except Exception as e:
    print(e)


HTTPSConnectionPool(host='read.douban.com', port=443): Max retries exceeded with url: /provider/all (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018B6E927D90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')))


In [1]:
import sys
import numpy as np
import json
import re
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import os
headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    }
url='https://read.douban.com/provider/all'                     
cookie={'cookie':'bid=QhkSDkC-6D4; ll="108235"; __utma=30149280.1604062177.1679555773.1679555773.1679555773.1; __utmz=30149280.1679555773.1.1.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __gads=ID=8a3e8dcdd0ebd524-22aef64e8edc0077:T=1679555778:RT=1679555778:S=ALNI_MYat1Lbo2a4rKijb4czFgslEoxqvw; _ga=GA1.3.1604062177.1679555773; _ga=GA1.1.1604062177.1679555773; __gpi=UID=00000bdebf1cdfd9:T=1679555778:RT=1679633581:S=ALNI_Mb7pC52M6-drvqg3ia2C6iRFXIesA; apiKey=; dbcl2="215291240:BgbV5x3+mU4"; ck=Jcux; _ga_RXNMP372GL=GS1.1.1695114097.4.0.1695114097.60.0.0; _gid=GA1.3.1870514339.1695114098; _pk_ref.100001.a7dd=%5B%22%22%2C%22%22%2C1695114098%2C%22https%3A%2F%2Fopen.weixin.qq.com%2F%22%5D; _pk_id.100001.a7dd=58088c42b0c59ce5.1679555779.; _pk_ses.100001.a7dd=1'}
try:
    response = requests.get(url,headers=headers,cookies=cookie)
    #将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
    soup = BeautifulSoup(response.text,'lxml')  
    
    #返回所有的<div>所有标签
    publishes = soup.find_all('div',{'class':'provider-group'})
    pbs=[]
    item_list=[]
    #enumerate爬虫中的遍历
    #遍历所有出版社
    #pb为当前出版社
    for index,pb in enumerate(publishes):
        #if (index<=1):
        if True:
            pb_list={}
            pb_list['item_name']=pb.find_next('div').text
            p=pb.find_next('ul')
            li_s=p.find_all('li')#li_s存储了当前出版社的所有数据。
            
        #print(li_s)
        book_name=[]
        book_value=[]
        for li in li_s:
            item_li={}
            #item_li为进入当前出版社内页书单的链接
            item_li['href']='https://read.douban.com'+li.find_next('a').get('href')

            url2=item_li['href']
            response2=requests.get(url2,headers=headers,cookies=cookie)
            soup2 = BeautifulSoup(response2.text,'lxml')  
            
            #遍历当前出版社的所有书单页面
            while soup2.find('li',class_='next')!=None:
                #booklist为当前页面的所有<div class=info>的书籍数据
                booklist=soup2.find_all('div',{'class':'info'})
                #print(publishes2)
                #遍历当前页面的所有书籍
                #book为当前的书籍数据
                for book in booklist:
                    
                    if(book.find('h4',class_='title')==None):
                        continue
                    title =book.find('h4',class_='title').text
                    item_li['name']=title
                    if(book.find('div',class_='sales-price')!=None):
                        price=book.find('div',class_='sales-price').text
                        item_li['price']=price
                    elif(book.find('span',class_='discount-price')!=None):
                        price=book.find('span',class_='discount-price').text
                        item_li['price']=price
                    elif(book.find('span',class_='price-tag')!=None):
                        price=book.find('span',class_='price-tag').text
                        item_li['price']=price
                    else:
                        continue
                    #输出查看
                    print(f"《{title}》:{price}")#1000行截断，保存成xlsx比较好
                    #item_list用来存储要求得的书名和价格的list型数据结构，一维
                    item_list.append([title,price])
                temp2=soup2.find('li',class_='next')
                #若存在后页
                if temp2.find('a')!=None:
                    #跳转到下一页
                    url3=url2+temp2.find('a').get('href')
                    response2=requests.get(url3,headers=headers,cookies=cookie)
                    soup2 = BeautifulSoup(response2.text,'lxml') 
                else:
                    break
                
    df=pd.DataFrame(item_list)
    df.columns=['书籍名称','价格']
    print(df)
    #保存到excel文件中
    df.to_excel("爬虫数据.xlsx")

    json_data = json.loads(str(pbs).replace("\'","\"").replace("\\xa0",""))  
    with open('work/' + 'pbs.json', 'w', encoding='UTF-8') as f:
        json.dump(json_data, f, ensure_ascii=False)
except Exception as e:
    print(e)


《云边有个小卖部》:13.99元
《病隙碎笔》:9.99元
《你也走了很远的路吧（增订本）》:29.99元
《亲密关系（全3册）》:48.97元
《苏东坡传》:9.99元
《我与地坛（插图版）》:9.99元
《中国通史》:2.00元
《通透》:62.00元
《唐诗生死局（全二册）》:61.99元
《日本蜡烛图技术（第2版）》:9.99元
《有钱人和你想的不一样（新版）》:12.49元
《中华帝国的衰落》:9.90元
《紫金陈：高智商犯罪（套装全4册）》:94.99元
《亲密关系：通往灵魂的桥梁》:16.99元
《南渡北归·南渡（上）》:11.99元
《山茶文具店》:10.99元
《赛雷三分钟漫画史（套装全7册）》:127.89元
《邻居》:40.99元
《地球无应答》:19.98元
《秘密》:17.99元
《香水之书》:63.99元
《沈氏女科：600年家传秘方大公开（套装全3册）》:55.97元
《官路》:17.99元
《罗马史纲》:39.99元
《博弈论的诡计》:25.99元
《南渡北归（套装全六册）》:69.99元
《人间草木》:9.99元
《人性的弱点（增订全译版）》:17.99元
《天堂旅行团》:29.99元
《鲜衣怒马少年时》:9.99元
《紫金陈：推理之王系列（共3册）》:99.99元
《古人很潮·第一季：古代名士、美男、网红等绝密档案（套装共5册）》:137.95元
《郭论（套装全6册）》:159.75元
《简读中国史：世界史坐标下的中国》:11.99元
《告白》:9.90元
《寻找百忧解》:41.99元
《予你长生》:34.99元
《生而为人》:39.99元
《成何体统（全2册）》:54.99元
《龙在宇商战小说集（套装全7册）》:94.40元
《了不起的心理学》:29.00元
《高智商犯罪1·设局》:20.00元
《惩罚者（套装全3册）》:74.58元
《让我留在你身边》:14.70元
《大城市的兴衰》:45.49元
《岳南作品集：一代大师们的经典重现（套装共6册）》:99.91元
《少年前传》:48.99元
《挣脱：一个律师的女性辩护记录》:30.79元
《春祺夏安》:26.59元
《九滴水·尸案调查科系列（套装全7册）》:124.99元
《树犹如此》:17.99元
《观山海》:17.99元
《偷影子的人 》:9.90元
《生活蒙太奇》:25.

KeyboardInterrupt: 